In [36]:
import os
import numpy as np
import glob
import pandas as pd

from astropy.table import Table, Column, vstack
import xspec

In [7]:
from astroquery.esa.xmm_newton import XMMNewton 

Created TAP+ (v20200428.1) - Connection:
	Host: nxsa.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


In [13]:
XMMNewton.download_data('0505720401',level="ODF",instname="M1",filename="result0505720401.tar") 

In [2]:
from astropy.io import fits
import matplotlib.pylab as plt
home= os.path.expanduser("-")

In [3]:
plt.rc("text", usetex=False)
plt.rc('font', family="serif")

In [4]:
!ls

4xmmdr11slim_210716.fits   init_sas.sh		       sas_docker_notebook.sh
Dockerfile		   pfiles		       sas_singularity.sh
Makefile		   pip.conf		       self-test.sh
README.md		   requirements.txt	       tests
Untitled.ipynb		   sas_18.0.0.conf	       xspec.log
User-Makefile		   sas_19.1.0-Ubuntu18.04.tgz
ds9.ubuntu20.8.2.1.tar.gz  sas_docker.sh


In [42]:
logfile = xspec.Xset.openLog(f"xspec.log")

Will no longer write to previously opened log file.
Logging to file: xspec.log


In [43]:
!ls

Dockerfile		   omgrismspec.rsp	       sas_docker.sh
Makefile		   pfiles		       sas_docker_notebook.sh
README.md		   pip.conf		       sas_singularity.sh
Untitled.ipynb		   requirements.txt	       self-test.sh
User-Makefile		   response.rsp		       startsas.log
ds9.ubuntu20.8.2.1.tar.gz  result0505720401.tar.gz     tests
init_sas.sh		   sas-startup.ipynb	       xspec.log
obs			   sas_18.0.0.conf
omgrismspec.pi		   sas_19.1.0-Ubuntu18.04.tgz


In [109]:
s = xspec.Spectrum("obs/obs/P0104860501R2S005SRSPEC1001.FIT")


6 spectra  in use
 
Spectral Data File: obs/obs/P0104860501R2S005SRSPEC1001.FIT  Spectrum 6
Net count rate (cts/s) for Spectrum:6  4.587e-02 +/- 1.068e-03
 Assigned to Data Group 1 and Plot Group 6
  Noticed Channels:  1-2714
  Telescope: XMM Instrument: RGS2  Channel Type: PI
  Exposure Time: 4.018e+04 sec
 Using fit statistic: cstat
 No response loaded.

***Warning!  One or more spectra are missing responses,
               and are not suitable for fit.


In [110]:
model = xspec.Model("bbody + ga")


Model bbody<1> + gaussian<2> Source No.: 1   Active/Off
Model Model Component  Parameter  Unit     Value
 par  comp
   1    1   bbody      kT         keV      3.00000      +/-  0.0          
   2    1   bbody      norm                1.00000      +/-  0.0          
   3    2   gaussian   LineE      keV      6.50000      +/-  0.0          
   4    2   gaussian   Sigma      keV      0.100000     +/-  0.0          
   5    2   gaussian   norm                1.00000      +/-  0.0          
________________________________________________________________________

***Warning!  One or more spectra are missing responses,
               and are not suitable for fit.


In [111]:
ncomp = len(model.componentNames)
for icomp in model.componentNames:
    print(icomp, eval(f"model.{icomp}.parameterNames"))

bbody ['kT', 'norm']
gaussian ['LineE', 'Sigma', 'norm']


In [112]:
#
# the BlackBody peak at 6.5  keV 
#
# Wien's law, the temperature in kT units [keV] of a BB peaking at 6.5 keV
#
model.bbody.kT = 6.5/1.59362
# and fix it
model.bbody.kT.frozen = True
#
# initial sigma of the line is set to 50 eV, but can be between 1 and 100 eV
model.gaussian.Sigma = [5.0e-2,0.001,1.0e-3,1.0e-3,1.0e-1,1.0e-1]
#
line_c = 6.4 # keV
# set the initial line energy and the range where to lok for a C-stat minimum
model.gaussian.LineE = [line_c,0.001,line_c-0.2,line_c-0.1,line_c+0.1,line_c+0.2]

***Warning!  One or more spectra are missing responses,
               and are not suitable for fit.
***Warning!  One or more spectra are missing responses,
               and are not suitable for fit.
***Warning!  One or more spectra are missing responses,
               and are not suitable for fit.
***Warning!  One or more spectra are missing responses,
               and are not suitable for fit.


In [113]:
#
xspec.Fit.statMethod = "cstat"
xspec.Xset.abund = "wilm"
#
#
# will constrain the spectrum in [4.5-8] keV
#
startE = 4.5 # keV
endE = 8.0 # keV
#
s.notice("all")
s.ignore(f"*-{startE} {endE}-*")
#

Default fit statistic is set to: C-Statistic
   This will apply to all current and newly loaded spectra.
***Warning!  One or more spectra are missing responses,
               and are not suitable for fit.
 Solar Abundance Vector set to wilm:  Wilms, J., Allen, A. & McCray, R. ApJ 542 914 (2000) (abundances are set to zero for those elements not included in the paper).
***Warning!  One or more spectra are missing responses,
               and are not suitable for fit.
  2714 channels (1,2714) noticed in spectrum #     6
***Warning!  One or more spectra are missing responses,
               and are not suitable for fit.





***XSPEC Error:  no energy defined range for spectrum 6

***XSPEC Error:  no energy defined range for spectrum 6


In [114]:
model.show()


Model bbody<1> + gaussian<2> Source No.: 1   Active/Off
Model Model Component  Parameter  Unit     Value
 par  comp
   1    1   bbody      kT         keV      4.07876      frozen
   2    1   bbody      norm                1.00000      +/-  0.0          
   3    2   gaussian   LineE      keV      6.40000      +/-  0.0          
   4    2   gaussian   Sigma      keV      5.00000E-02  +/-  0.0          
   5    2   gaussian   norm                1.00000      +/-  0.0          
________________________________________________________________________



In [115]:
xspec.Fit.nIterations = 100
xspec.Fit.query = 'yes'
xspec.Fit.perform()
xspec.Fit.show()
cstat = xspec.Fit.statistic
dof = xspec.Fit.dof
chi2r = xspec.Fit.testStatistic/dof
#xspec.Fit.error("2.706 3 6")


***XSPEC Error: Cannot fit: No models are currently active.


Exception: Error executing command: fit

In [116]:
# plot in Xserver
xspec.Plot.device = '/xs'
# or save to postScript
#xspec.Plot.device = f'{psfile}/cps'
xspec.Plot.xAxis = "keV"
xspec.Plot.setRebin(minSig=30.0,maxBins=4)
xspec.Plot.addCommand("csize 1.2")
xspec.Plot.addCommand("lwidth 4")
xspec.Plot.addCommand("lab top \"Test plot\"")
xspec.Plot('ld','ratio')


***XSPEC Error: No energies defined for spectrum 1, but plot requested on energy/wavelength axis.
 Plot Group 1 will not be constructed 



Exception: 

In [117]:
!ls obs

0104860501  obs


In [52]:
os.getcwd()

'/home/heasoft'

In [121]:
!rm omgrismspec.pi
!rm omgrismspec.rsp
!omgrismresp inputspec=obs/obs/P0104860501R2S005SRSPEC1001.FIT

rm: cannot remove 'omgrismspec.pi': No such file or directory
rm: cannot remove 'omgrismspec.rsp': No such file or directory
omgrismresp:- Executing (routine): omgrismresp inputspec=obs/obs/P0104860501R2S005SRSPEC1001.FIT outputspec=omgrismspec.pi responsefile=omgrismspec.rsp  -w 1 -V 4
omgrismresp:- omgrismresp (omgrismresp-0.5.1)  [xmmsas_20210317_1624-19.1.0] started:  2021-11-12T03:11:05.000
  Error: Filter UNKNOWN              not recognised
omgrismresp:- omgrismresp (omgrismresp-0.5.1)  [xmmsas_20210317_1624-19.1.0] ended:    2021-11-12T03:11:05.000


In [118]:
xspec.Plot.commands = ()

Plot command list is now empty


In [119]:
save_data = 'specfit.qdp'
if (os.path.isfile(save_data)):
    os.remove(save_data)
# following Andy Beardmore idea
xspec.Plot.device = '/null'
xspec.Plot.add = True
xspec.Plot.addCommand(f'wd {save_data}')
xspec.Plot("ld")


***XSPEC Error: No energies defined for spectrum 1, but plot requested on energy/wavelength axis.
 Plot Group 1 will not be constructed 



Exception: 


***XSPEC Error: Cannot fit: No models are currently active.


Exception: Error executing command: fit